<a href="https://colab.research.google.com/github/danielemurgolo/FantasyF1/blob/main/FantasyF1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
!pip install pulp

In [156]:
from pulp import *
import pandas as pd
import numpy as np

#change Hulkenberg->Vettel

Drivers = ["Hamilton", "Verstappen", "Russell", "Leclerc", "Perez", "Sainz", "Norris", "Ricciardo",
    "Gasly", "Alonso", "Ocon", "Hulkenberg", "Stroll", "Bottas", "Tsunoda", "Guanyu", "Albon",
    "Latifi", "Schumacher", "Magnussen"]

Driver_dict = {
    'Mercedes' : ['Hamilton','Russell'],
    'RedBull' : ['Verstappen', 'Perez'],
    'Ferrari' : ['Leclerc', 'Sainz'],
    'McLaren' : ['Norris', 'Ricciardo'],
    'Alpine' : ['Alonso', 'Ocon'],
    'AstonMartin': ['Hulkenberg', 'Stroll'],
    'AlphaTauri' : ['Gasly', 'Tsunoda'],
    'AlphaRomeo': ['Bottas', 'Guanyu'],
    'Williams': ['Albon', 'Latifi'],
    'Haas' : ['Schumacher', 'Magnussen'] 
}

Teams = ['Mercedes', 'RedBull', 'Ferrari', 'McLaren', 'Alpine', 'AstonMartin', 'AlphaTauri', 'AlphaRomeo', 'Williams', 'Haas']

Drivers_Importance = np.array([5, 2, 9, 1, 4, 3, 13, 18, 10, 8, 11, 17, 19, 6, 16, 15, 14, 20, 12, 7])

Teams_Importance = np.array([7, 3, 1.5, 15.5, 9.5, 18, 13, 10.5, 17, 9.5])

Teams_cost = np.array([34.5, 32.5, 25.0, 18.5, 14.0, 11.5, 10.5, 8, 7, 6])


Drivers_cost = np.array([31, 30.5, 24, 18, 17.5, 17, 16, 14.5, 13.5, 12.5, 12, 11.5, 9.5, 9, 8.5, 8, 7.5, 7, 6.5, 5.5])

model = LpProblem("Fantasy F1 problem", LpMinimize)




/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [157]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from math import sqrt

In [158]:
#url of fp3
url = 'https://www.formula1.com/en/results.html/2022/races/1124/bahrain/practice-2.html'

#Create object page
page = requests.get(url)

# parser-lxml = Change html to Python friendly format
# Obtain page's information
soup = BeautifulSoup(page.text, 'lxml')

In [159]:
# Obtain information from tag <table>
table = soup.find('table', class_ = 'resultsarchive-table')

In [160]:
# Obtain every title of columns with tag <th>
headers = []
for i in table.find_all('th'):
 title = i.text
 headers.append(title)

# Create a dataframe
fp3table = pd.DataFrame(columns = headers)

In [161]:
# Create a for loop to fill mydata
for j in table.find_all('tr')[1:]:
 row_data = j.find_all('td')
 row = [i.text for i in row_data]
 length = len(fp3table)
 fp3table.loc[length] = row

In [162]:
import re

pattern = r'\n+\w+\n'

In [163]:
def time_to_microseconds(time):
  minutes = time.minute
  seconds = time.second
  microseconds = time.microsecond

  return (minutes*6e+7)+(seconds*1e+6)+(microseconds)

for driver in fp3table['Driver']:
  fp3table['Driver'] = fp3table['Driver'].replace(driver , (re.sub(pattern, '', driver)))

for i,time in enumerate(fp3table['Time']):
  lap = datetime.strptime(time, '%M:%S.%f').time()
  driver = fp3table['Driver'][i]
  index = Drivers.index(driver)
  Drivers_Importance[index] = time_to_microseconds(lap)

for i,team in enumerate(Teams):
  driver_1, driver_2 = Driver_dict.get(team)
  index_1 = Drivers.index(driver_1)
  index_2 = Drivers.index(driver_2)
  Teams_Importance[i] = sqrt(Drivers_Importance[index_1]*Drivers_Importance[index_2])

In [164]:
drivers_decision_variables = LpVariable.matrix('X', Drivers, cat = "Binary", lowBound= 0)


teams_decision_variables = LpVariable.matrix('X', Teams, cat= 'Binary', lowBound=0)

In [165]:
driver_func = lpSum(drivers_decision_variables*Drivers_Importance)

team_func = lpSum(teams_decision_variables*Teams_Importance)

obj_func = driver_func + team_func

model +=  obj_func
print(model)

Fantasy_F1_problem:
MINIMIZE
94735000*X_Albon + 92877000*X_Alonso + 93450657.04958954*X_AlphaRomeo + 93704962.3499204*X_AlphaTauri + 93118186.83801785*X_Alpine + 94009485.8937118*X_AstonMartin + 92951000*X_Bottas + 92271165.37683915*X_Ferrari + 93621000*X_Gasly + 93953000*X_Guanyu + 93133987.10996969*X_Haas + 93144000*X_Hamilton + 94061000*X_Hulkenberg + 94486000*X_Latifi + 92023000*X_Leclerc + 93183000*X_Magnussen + 93721953.03129359*X_McLaren + 92835990.73635182*X_Mercedes + 93280000*X_Norris + 93360000*X_Ocon + 92958000*X_Perez + 92445587.71515274*X_RedBull + 94166000*X_Ricciardo + 92529000*X_Russell + 92520000*X_Sainz + 93085000*X_Schumacher + 93958000*X_Stroll + 93789000*X_Tsunoda + 91936000*X_Verstappen + 94610418.08384529*X_Williams + 0.0
VARIABLES
0 <= X_Albon <= 1 Integer
0 <= X_Alonso <= 1 Integer
0 <= X_AlphaRomeo <= 1 Integer
0 <= X_AlphaTauri <= 1 Integer
0 <= X_Alpine <= 1 Integer
0 <= X_AstonMartin <= 1 Integer
0 <= X_Bottas <= 1 Integer
0 <= X_Ferrari <= 1 Integer
0 <= 

In [166]:
driver_budget = lpSum(drivers_decision_variables*Drivers_cost)
team_budget = lpSum(teams_decision_variables*Teams_cost)
model += driver_budget + team_budget <= 100 , 'Budget'

In [167]:
model += lpSum(drivers_decision_variables) == 5, 'N of Drivers'

In [168]:
model += lpSum(teams_decision_variables) == 1, 'N of Team'

In [169]:
print(model)

Fantasy_F1_problem:
MINIMIZE
94735000*X_Albon + 92877000*X_Alonso + 93450657.04958954*X_AlphaRomeo + 93704962.3499204*X_AlphaTauri + 93118186.83801785*X_Alpine + 94009485.8937118*X_AstonMartin + 92951000*X_Bottas + 92271165.37683915*X_Ferrari + 93621000*X_Gasly + 93953000*X_Guanyu + 93133987.10996969*X_Haas + 93144000*X_Hamilton + 94061000*X_Hulkenberg + 94486000*X_Latifi + 92023000*X_Leclerc + 93183000*X_Magnussen + 93721953.03129359*X_McLaren + 92835990.73635182*X_Mercedes + 93280000*X_Norris + 93360000*X_Ocon + 92958000*X_Perez + 92445587.71515274*X_RedBull + 94166000*X_Ricciardo + 92529000*X_Russell + 92520000*X_Sainz + 93085000*X_Schumacher + 93958000*X_Stroll + 93789000*X_Tsunoda + 91936000*X_Verstappen + 94610418.08384529*X_Williams + 0.0
SUBJECT TO
Budget: 7.5 X_Albon + 12.5 X_Alonso + 8 X_AlphaRomeo + 10.5 X_AlphaTauri
 + 14 X_Alpine + 11.5 X_AstonMartin + 9 X_Bottas + 25 X_Ferrari + 13.5 X_Gasly
 + 8 X_Guanyu + 6 X_Haas + 31 X_Hamilton + 11.5 X_Hulkenberg + 7 X_Latifi
 + 18 X

In [170]:
model.solve(PULP_CBC_CMD())

status =  LpStatus[model.status]

print(status)

Optimal


In [171]:
for v in model.variables():
  if v.value() !=0:
    print(v.name)

X_Alonso
X_Ferrari
X_Leclerc
X_Magnussen
X_Schumacher
X_Verstappen
